# LSTM Stock Price Predictor — Experiments

Interactive notebook for exploring the LSTM prediction pipeline.

**Sections:**
1. Setup & Imports
2. Data Exploration
3. Preprocessing Visualization
4. Single Training Run
5. Predictions & Diagnostics
6. Baseline Comparison
7. Walk-Forward Validation
8. Hyperparameter Experiments

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from app.ml.providers.mock_provider import MockDataProvider
from app.ml.models.schemas import TrainingConfig
from app.ml.preprocessing.pipeline import DataPipeline
from app.ml.preprocessing.scaler import PriceScaler
from app.ml.preprocessing.windowing import create_sequences
from app.ml.training.lstm_model import build_lstm_model
from app.ml.training.baseline import PersistenceBaseline
from app.ml.evaluation.metrics import (
    calculate_rmse, calculate_mae, calculate_mape, calculate_directional_accuracy
)
from app.ml.services.prediction_service import PredictionService

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
print('Setup complete!')

## 1. Data Exploration

Fetch data using MockDataProvider (deterministic sine wave) or PolygonDataProvider (real market data).

In [ ]:
# Use mock data (no API key needed)
provider = MockDataProvider(seed=42)
raw = provider.fetch_ohlcv('AAPL', '2022-01-01', '2024-01-01')

df = pd.DataFrame(raw)
df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
df = df.sort_values('timestamp').reset_index(drop=True)

print(f'Rows: {len(df)}')
print(f'Date range: {df["date"].min()} to {df["date"].max()}')
df.describe()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

axes[0].plot(df['date'], df['close'], linewidth=1)
axes[0].set_title('Close Price')
axes[0].set_ylabel('Price ($)')

axes[1].bar(df['date'], df['volume'], width=1, alpha=0.6)
axes[1].set_title('Volume')
axes[1].set_ylabel('Volume')
axes[1].set_xlabel('Date')

plt.tight_layout()
plt.show()

## 2. Preprocessing Visualization

In [ ]:
# Scale the close prices
close_data = df[['close']].values.astype(np.float64)
scaler = PriceScaler()
scaled = scaler.fit_transform(close_data)

print(f'Original range: [{close_data.min():.2f}, {close_data.max():.2f}]')
print(f'Scaled range:   [{scaled.min():.4f}, {scaled.max():.4f}]')

# Verify inverse transform
recovered = scaler.inverse_transform(scaled)
roundtrip_error = np.max(np.abs(close_data - recovered))
print(f'Inverse transform max error: {roundtrip_error:.10f}')

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].plot(close_data, label='Original')
axes[0].set_title('Original Close Prices')
axes[1].plot(scaled, label='Scaled', color='orange')
axes[1].set_title('Scaled [0, 1]')
plt.tight_layout()
plt.show()

In [ ]:
# Create sequences and visualize first window
SEQ_LEN = 60
X, y = create_sequences(scaled, SEQ_LEN)
print(f'X shape: {X.shape}  (samples, timesteps, features)')
print(f'y shape: {y.shape}')

# Plot first window vs its target
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(range(SEQ_LEN), X[0, :, 0], label='Input window (60 days)')
ax.scatter([SEQ_LEN], [y[0]], color='red', s=100, zorder=5, label='Target')
ax.set_title('First Sequence Window → Target')
ax.set_xlabel('Time Step')
ax.legend()
plt.show()

## 3. Single Training Run

In [ ]:
config = TrainingConfig(
    ticker='MOCK',
    from_date='2022-01-01',
    to_date='2024-01-01',
    epochs=30,
    sequence_length=60,
    batch_size=32,
    features=['close'],
)

service = PredictionService(provider)
result, test_pred, y_test, history = service.train(config)

print(f'\nVal RMSE:       {result.val_rmse}')
print(f'Baseline RMSE:  {result.baseline_rmse}')
print(f'Improvement:    {result.improvement_over_baseline}%')
print(f'Best Epoch:     {result.best_epoch}/{result.epochs_completed}')

## 4. Predictions & Diagnostics

In [ ]:
# Training curves
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(history['loss'], label='Training Loss')
ax.plot(history['val_loss'], label='Validation Loss')
ax.set_title('Training History')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss (MSE)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Actual vs Predicted
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(y_test, label='Actual', linewidth=1.5)
ax.plot(test_pred, label='Predicted', linewidth=1.5, alpha=0.8)
ax.set_title('Actual vs Predicted (Test Set)')
ax.set_xlabel('Time Steps')
ax.set_ylabel('Price (scaled)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

# Zoom in on last 60 steps
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(y_test[-60:], label='Actual', marker='o', markersize=3)
ax.plot(test_pred[-60:], label='Predicted', marker='x', markersize=3, alpha=0.8)
ax.set_title('Zoom: Last 60 Steps')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Residuals
residuals = y_test - test_pred
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(residuals, alpha=0.6)
axes[0].axhline(y=0, color='r', linestyle='--')
axes[0].set_title('Residuals Over Time')

axes[1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[1].set_title('Residual Distribution')

plt.tight_layout()
plt.show()

print(f'Residual mean:  {residuals.mean():.6f}')
print(f'Residual std:   {residuals.std():.6f}')

## 5. Baseline Comparison

In [ ]:
baseline_pred = PersistenceBaseline.predict(y_test)

lstm_rmse = calculate_rmse(y_test, test_pred)
baseline_rmse = calculate_rmse(y_test, baseline_pred)

lstm_dir_acc = calculate_directional_accuracy(y_test, test_pred)
baseline_dir_acc = calculate_directional_accuracy(y_test, baseline_pred)

print(f'LSTM      — RMSE: {lstm_rmse:.6f}, Dir.Acc: {lstm_dir_acc:.1f}%')
print(f'Baseline  — RMSE: {baseline_rmse:.6f}, Dir.Acc: {baseline_dir_acc:.1f}%')
print(f'Improvement: {((baseline_rmse - lstm_rmse) / baseline_rmse) * 100:.2f}%')

## 6. Walk-Forward Validation

In [ ]:
wf_config = TrainingConfig(
    ticker='MOCK',
    from_date='2022-01-01',
    to_date='2024-01-01',
    epochs=10,
    sequence_length=30,
    features=['close'],
)

wf_result = service.validate(wf_config, n_folds=3)

print(f'Folds: {wf_result.num_folds}')
print(f'Avg RMSE: {wf_result.avg_rmse}')
print(f'Avg Dir.Acc: {wf_result.avg_directional_accuracy}%')
print()
for fold in wf_result.fold_results:
    print(f"  Fold {fold['fold']}: RMSE={fold['rmse']:.6f}, Dir.Acc={fold['directional_accuracy']:.1f}%")

## 7. Hyperparameter Experiments

Experiment with different sequence lengths, LSTM units, and features.

In [ ]:
# Experiment: different sequence lengths
results = []
for seq_len in [20, 40, 60]:
    exp_config = TrainingConfig(
        ticker='MOCK',
        from_date='2022-01-01',
        to_date='2024-01-01',
        epochs=15,
        sequence_length=seq_len,
        features=['close'],
    )
    exp_result, _, _, _ = service.train(exp_config)
    results.append({'seq_len': seq_len, 'val_rmse': exp_result.val_rmse, 'improvement': exp_result.improvement_over_baseline})
    print(f'seq_len={seq_len}: val_RMSE={exp_result.val_rmse}, improvement={exp_result.improvement_over_baseline}%')

results_df = pd.DataFrame(results)
results_df.plot(x='seq_len', y='val_rmse', kind='bar', title='RMSE by Sequence Length')
plt.show()